# Welcome to the Santa 2021 - Movie Madness! #

Each year Kaggle holds a Christmas-themed optimization challenge. This year, your challenge is to optimize the scheduling of seven movies.

This problem can be stated like this:

Find a set of three strings containing every permutation of the seven symbols 🎅, 🤶, 🦌, 🧝, 🎄, 🎁, and 🎀 as substrings, subject to the following conditions:

- Every permutation must be in at least one string.
- Each permutation beginning with 🎅🤶 must be in all three strings.
- Each string may have up to two wildcards 🌟, which will match any symbol in a permutation. No string of length seven containing more than one wildcard will count as a permutation.
- Your score is the length of the longest of the three strings, with lower scores better.

See this competition's [Evaluation](https://www.kaggle.com/c/santa-2021/overview/evaluation) page for more details.

In this notebook, we'll see how the elves' movie-scheduling problem can be formulated as a type of **minimal superpermutation problem** and also as a type of **traveling salesman** or **routing problem**. We'll implement this latter formulation in Google's open-source [OR-Tools](https://developers.google.com/optimization) optimization library to create a submission.

# The Minimal Superpermutation Problem #

A **permutation** over a set of symbols is a particular arrangement of those symbols. For instance, there are six permutations of the symbols 1, 2, and 3: 123, 132, 213, 231, 312, 321. Notice that (allowing overlap) these permutations can be packed together very efficiently into longer strings. The permutations 123 and 231, for example, are both substrings of 1231, a string of length four. Extending this by one character to 12312, we can include a third permutation, 312. In fact, *every* permutation over 1, 2, 3 can be found within a string of length 9, the string 123121321.

Strings that contain every permutation of some set of symbols are called **superpermutations**. The **minimal superpermutation problem** asks: how short can superpermutations actually be? Exact answers are known for $ n \leq 5 $ symbols, but for $ n \geq 6 $ researchers have resorted to discovering superpermutations through various search and optimization techniques.

We can see now that the elves' scheduling problem is a variation on this minimal superpermutation problem, with three strings instead of one and the addition of a "wildcard" symbol. A lot of work has been done on superpermutations already, which you might like to use for inspiration. Check out [Greg Egan's Superpermutation page](https://www.gregegan.net/SCIENCE/Superpermutations/Superpermutations.html) for a great introduction to current methods.

# The Traveling Salesman Formulation #

By defining the notion of a distance on permutations, we can view the superpermutation and movie scheduling problems as kinds of traveling salesman or [routing](https://developers.google.com/optimization/routing) problems.

We saw in the last section how we could add a single character to 123 to get a string also containing 231 (the string 1231). This tells us that the *distance* from 123 to 231 is 1 (one character). More generally, we define the **distance** from a permutation p to a permutation q by:

$$ d(p, q) = \text{number of characters you have to append to } p \text{ to get a string also containing } q $$

which in Python we could implement as:

``` python
def distance(p, q):
    N = len(p)
    for n in range(N+1):
        if p[n:] == q[:N-n]:
            return n
```

This function essentially measures the deficiency of overlap from the end of p to the beginning of q. We have, for example,

``` python
distance(123, 123) == 0
distance(123, 231) == 1  # with 1231
distance(123, 321) == 2  # with 12321
distance(123, 132) == 3  # with 123132
```

Now, if you imagine the permutations as nodes in a graph connected by these distances, the minimal superpermutation problem becomes the problem of finding the shortest route through this graph that connects every node, the Traveling Salesman Problem. Note, however, that our permutation distance function is *not* symmetric: `distance(123, 231) == 1`, but `distance(231, 123) == 2`. This makes the optimization problem somewhat harder than a traditional TSP.

This graph illustrates the route generating the superpermutation 123121321.

In [ ]:
from graphviz import Digraph

m = Digraph(
    'tsp_graph', 
    graph_attr={
        'fontname': 'sans'
    }, 
    node_attr={
        'shape': 'circle',
        'fontname': 'sans'
    },
    edge_attr={
        'fontname': 'sans',
    },
    engine='sfdp',
    directory='images',
    filename='tsp_graph',
    format='png',
)

with m.subgraph(name='cluster_1') as layer:
    layer.attr(color='invis')
    layer.node('123')
    layer.node('231')
    layer.node('321')

with m.subgraph(name='cluster_2') as layer:
    layer.attr(color='invis')
    layer.node('132')
    layer.node('321')
    layer.node('213')

m.edge('123', '231', label='1')
m.edge('231', '312', label='1')
m.edge('312', '213', label='2')
m.edge('213', '132', label='1')
m.edge('132', '321', label='1')

m.edge('312', '123', color='invis')
m.edge('321', '213', color='invis')

m.render()
m

The movie scheduling problem is similar, except we have multiple routes with a subset of nodes being visited by every salesman.

Kaggle has held TSP challenges in previous years. You might like to review the discussion or notebooks of these competitions for inspiration:
- [Traveling Santa 2018 - Prime Paths](https://www.kaggle.com/c/traveling-santa-2018-prime-paths/overview)
- [Traveling Santa Problem](https://www.kaggle.com/c/traveling-santa-problem)

# Solving the Problem with OR-Tools #

We'll implement a solver for the movie-scheduling problem with the [Vehicle Routing](https://developers.google.com/optimization/routing/vrp) module of OR-Tools.

First we'll define our imports and set some configuration variables.

In [ ]:
import functools
import itertools
import math

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


# Configure the problem
N_STRINGS = 3
LETTERS = [
    '🎅',  # father christmas
    '🤶',  # mother christmas
    '🦌',  # reindeer
    '🧝',  # elf
    '🎄',  # christmas tree
    '🎁',  # gift
    '🎀',  # ribbon
]
WILDCARD = '🌟'  # star
N = len(LETTERS)
DEPOT = itertools.repeat('0', N)  # a starting dummy node
INF = 999  # number to represent "infinite distance", could try sys.maxsize for hard constraint

# Configure stopping conditions for the search
N_SOLUTIONS = None
N_MINUTES = 60

Now we need to create the "data model" defining the nodes in the graph. The nodes in this case are the permutations themselves. To enforce the condition that permutations beginning with 🎅🤶 must be in every schedule, we create a set of such nodes for each schedule (each string of the solution, that is). The `DEPOT` is a dummy node we include to remove the condition that every string has to begin with a certain permutation.

In [ ]:
def make_nodes():
    perms = list(itertools.permutations(LETTERS, N))
    all_ = perms[:math.factorial(N-2)]  # permutations beginning with 🎅🤶
    some = perms[math.factorial(N-2):]  # everything else
    return [DEPOT] + (all_ * N_STRINGS) + some


def create_data_model():
    data = {}
    data['locations'] = make_nodes()
    data['num_vehicles'] = N_STRINGS
    data['depot'] = 0
    return data


data = create_data_model()
manager = pywrapcp.RoutingIndexManager(
    len(data['locations']),
    data['num_vehicles'],
    data['depot'],
)
routing = pywrapcp.RoutingModel(manager)

And we also define how we're measuring the distance between nodes. We've included the full distance matrix as a CSV file in the dataset for this competition, but for this notebook we'll just use the distance function directly.

Modifying the distance function is one way to change the constraints on a problem or to modify the search space. First, to make a "dummy" of our depot node, we define its distance to every node to be 0.

Second, we reduce the search space by setting the maximum distance between nodes to infinity (or actually to some large number). This change means we would never consider going from node 🎅🤶🦌🧝🎄🎁🎀 to node 🧝🎄🎁🎀🎅🤶🦌, since these nodes are separated by the maximum distance of 7. Since most nodes are separated with maximum distance, this makes the search problem much easier but doesn't seem to exclude any potentially good solutions. You might find other changes that could reduce the search space further.

In [ ]:
def distance(p, q):
    if p == DEPOT or q == DEPOT: return 0
    for n in range(N+1):  # never choose maximum distance nodes (the N+1 case)
        if p[n:] == q[:N-n]:
            return n
    return INF  # max distance N becomes distance infinity


def distance_callback(from_index, to_index):
    # Convert from the internal representation to an actual permutation
    from_node = manager.IndexToNode(from_index)
    from_perm = data['locations'][from_node]
    to_node = manager.IndexToNode(to_index)
    to_perm = data['locations'][to_node]
    return distance(from_perm, to_perm)


transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

By default, the optimizer only considers the total length of all strings when searching for a solution. To enforce our objective of finding a minimal length for the longest string, we create an additional *dimension*.

Dimensions model quantities accumulated along a route. We add a dimension here to accumulate lengths for each route (that is, movie schedule) and set a cost proportional to the difference between the largest length and the smallest. Adding this cost will encourage the optimizer to make the largest length as short as possible.

In [ ]:
# Define a Length dimension that records the lengths of each schedule
dimension_name = 'Length'
routing.AddDimension(
    transit_callback_index,
    0,  # slack is additional fixed cost at each node; not applicable here
    10**16, # max length per route; set to some large-enough number
    True,  # start with total length of 0
    dimension_name)
length_dimension = routing.GetDimensionOrDie(dimension_name)
length_dimension.SetGlobalSpanCostCoefficient(100) # total cost += 100 * (max_length - min_length)

# Set constraint that each vehicle must have all permutations beginning with 🎅🤶
n_all = math.factorial(N-2)
for vehicle in range(N_STRINGS):
    for node in range(n_all):
        routing.SetAllowedVehiclesForIndex(
            [vehicle],
            manager.NodeToIndex(1+vehicle*n_all+node),
        )

You could also try removing some of the edges of the permutation graph. The cell below illustrates how, though we won't actually remove any.

In [ ]:
def remove_forbidden_edges(forbidden_fn):
    n_removed = 0
    n_nodes = len(data['locations'])

    for from_index, to_index in itertools.product(range(n_nodes), range(n_nodes)):
        from_node = manager.IndexToNode(from_index)
        from_perm = data['locations'][from_node]
        to_node = manager.IndexToNode(to_index)
        to_perm = data['locations'][to_node]
        
        if routing.NextVar(from_index).Contains(to_index) and forbidden_fn(from_perm, to_perm):
#             print("Removing edge", from_index, from_perm, " -> ", to_index, to_perm)
            n_removed += 1
            routing.NextVar(from_index).RemoveValue(to_index)
    
    print("Removed", n_removed, "edges.")
    

# remove_forbidden_edges(
#     lambda p, q: distance(p, q) >= N
# )

There are a number of [routing options](https://developers.google.com/optimization/routing/routing_options) available to modify the search strategy, a few of which we've set here. You can also define the [initial routes](https://developers.google.com/optimization/routing/routing_tasks#setting-initial-routes-for-a-search) for a search -- useful if you've discovered some good starting solution through other means.

In [ ]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.first_solution_strategy = \
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

search_parameters.local_search_metaheuristic = \
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH

# Define stopping conditions
if N_SOLUTIONS is not None:
    search_parameters.solution_limit = N_SOLUTIONS
if N_MINUTES is not None:
    search_parameters.time_limit.seconds = N_MINUTES * 60
    
search_parameters.log_search = True

Now that we've defined the problem, we can run the optimization routine to search for a minimal solution. We've set the optimizer to run for an hour, but you could also choose the number of solutions to iterate through.

In [ ]:
solution = routing.SolveWithParameters(search_parameters)

## Create a Submission ##

There are a couple of steps we need to go through to create the final submitted schedules. First, we need to convert the OR-Tools representation of the solution to routes through the permutation graph. The `get_routes` function collects the optimal routes found into three lists of (integer) nodes, something like:

``` python
{
    0: [1, 2, 10, 16, ...],
    1: [1, 80, 2, 72, ...],
    2: [3, 9, 21, 12, ...],
}
```

Second, we need to convert these lists into "superpermutation" format, a long string of symbols. The function `route_to_schedule` converts each integer index into the corresponding permutation node and then squeezes them all together, something like:

``` python
[50, 2, 9, ...]
-> ['🧝🎄🎁🎀🎅🤶🦌', '🎅🤶🦌🎁🎀🧝🎄', '🤶🦌🎁🎀🧝🎄🎅', ...]
-> '🧝🎄🎁🎀🎅🤶🦌🎁🎀🧝🎄🎅...'
```

In [ ]:
from collections import defaultdict


def get_routes(data, manager, routing, solution):
    routes = defaultdict(list)
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        while not routing.IsEnd(index):
            idx_node = manager.IndexToNode(index)
            if idx_node != data['depot']:
                routes[vehicle_id].append(idx_node)
            index = solution.Value(routing.NextVar(index))
    return routes


def route_to_schedule(route, nodes):
    def overlap(a, b):
        return max(i for i in range(len(b)+1) if a.endswith(b[:i]))

    def squeeze(ws):
        return functools.reduce(lambda a, b: a + b[overlap(a, b):], ws)

    return squeeze(["".join(nodes[i]) for i in route])


def get_schedules(routes):
    words = [route_to_schedule(routes[vehicle_id], data['locations'])
             for vehicle_id in range(data['num_vehicles'])]
    return words

We can now create the submission file `submission.csv`.

In [ ]:
import pandas as pd


if solution:
    routes = get_routes(data, manager, routing, solution)
    words = get_schedules(routes)

    submission = pd.Series(words, name='schedule')
    submission.to_csv('submission.csv', index=False)

    display(submission)
    display(submission.apply(len).rename("Lengths"))